# Property Graph Index with Local Llama Model and Local File

In this notebook, we demonstrate how to create, query, and customize a Property Graph Index using a local Llama model and a local file.

In [ ]:
!pip install llama-index
!pip install llama-index-embeddings-ollama

## Setting Up the Environment

In [ ]:
import os
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

## Loading Documents

In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
#documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()

# Load documents from the local directory
print(f"Number of documents loaded: {len(documents)}")

## Constructing the Property Graph Index

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core import PropertyGraphIndex

# Create Property Graph Index
index = PropertyGraphIndex.from_documents(
    documents,
    llm=ollama_llm,
    embed_model=ollama_embedding,
    show_progress=True
)

## Exploring the Property Graph

In [ ]:
!pip install pyvis


In [ ]:

# Save and visualize the graph
index.property_graph_store.save_networkx_graph(name="./kg.html")
print("Graph saved as ./kg.html. Open it in a browser to explore.")

## Querying the Property Graph

In [ ]:
# Simple Retrieval
retriever = index.as_retriever(include_text=False)

# Query the graph
nodes = retriever.retrieve("What happened at Interleaf and Viaweb?")
for node in nodes:
    print(node.text)

In [ ]:
# Advanced Querying
query_engine = index.as_query_engine(include_text=True, llm=ollama_llm,)

response = query_engine.query("What happened at Interleaf and Viaweb?")
print(str(response))

## Persisting the Property Graph Index

In [ ]:
# Save the graph and embeddings for reuse
index.storage_context.persist(persist_dir="./property_graph_storage")

In [ ]:
# Configure Ollama Embedding
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

In [ ]:

from llama_index.core import StorageContext,ServiceContext, load_index_from_storage
from llama_index.core import Settings

# Set global settings
Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding


# Create a StorageContext
storage_context = StorageContext.from_defaults(
    persist_dir="./property_graph_storage"
)

# Load the index with service_context and storage_context
loaded_index = load_index_from_storage(
    storage_context=storage_context,
)

print("Index loaded successfully with ServiceContext!")


In [ ]:
# Simple Retrieval
retriever = loaded_index.as_retriever(include_text=False)

# Query the graph
nodes = retriever.retrieve("What happened at Interleaf and Viaweb?")
for node in nodes:
    print(node.text)